# Class Notebooks

* [HAPI_00.ipynb](HAPI_00.ipynb) - Introduction
* [HAPI_01.ipynb](HAPI_01.ipynb) - Basics
* [HAPI_02.ipynb](HAPI_02.ipynb) - Data structures
* [HAPI_03.ipynb](HAPI_03.ipynb) - Plotting
* **[HAPI_04.ipynb](HAPI_04.ipynb) - Problems (this Notebook)**

# Instructions

Use the rest of the class time to work on these problems. Each problem is expected to take at least one hour, so start with a problem that interests you.

You are welcome to work on problems not given here. However, priority for questions will be given to students working on these problems.

For in-person students, we encourage you to work with one or more neighbors on a problem.

Several HAPI experts are available for questions on chat, and three of us will be on-site for in-person questions.

# Working with Metadata I.

## Basic


Starting with

```python
import pickle as pickle
with open('data/availability.pkl', 'rb') as f:
    datasets = pickle.load(f)

print(datasets[0:3])
# [{'id': 'ace', 'title': 'ACE', 'startDate': '1997-08-25T17:48:00.000Z', 'stopDate': '2022-07-04T23:48:00.000Z'}, {'id': 'active', 'title': 'Active', 'startDate': '1989-09-29T00:00:00.000Z', 'stopDate': '1991-10-04T08:00:00.000Z'}, {'id': 'aec', 'title': 'AE-C', 'startDate': '1973-12-17T08:01:00.000Z', 'stopDate': '1978-12-10T00:00:00.000Z'}]
```

use the information in `datasets` to create a table and plot as described in the following subsections. You should be able to do this without requesting additional information from the server.

(The information in `datasets` was creating using `hapi()` [metadata calls](HAPI_02.ipynb#Metadata) to the SSCWeb HAPI server.)

### Table

Create a table showing the time interval of availability of ephemeris data from the SSCWeb HAPI server. The table should have the form

    ace              1997-08-25T17:48:00.000Z  2022-07-04T23:48:00.000Z
    active           1989-09-29T00:00:00.000Z  1991-10-04T08:00:00.000Z
    aec              1973-12-17T08:01:00.000Z  1978-12-10T00:00:00.000Z
    ... (243 more lines)

In [5]:
import pickle as pickle
with open('data/availability.pkl', 'rb') as f:
    datasets = pickle.load(f)
print(datasets[0:3])

# Enter your code here

[{'id': 'ace', 'title': 'ACE', 'startDate': '1997-08-25T17:48:00.000Z', 'stopDate': '2022-07-04T23:48:00.000Z'}, {'id': 'active', 'title': 'Active', 'startDate': '1989-09-29T00:00:00.000Z', 'stopDate': '1991-10-04T08:00:00.000Z'}, {'id': 'aec', 'title': 'AE-C', 'startDate': '1973-12-17T08:01:00.000Z', 'stopDate': '1978-12-10T00:00:00.000Z'}]


### Plot

Create a plot showing the time interval of availability of ephemeris data from the SSCWeb HAPI server. The plot should look similar to the following.

![](imgs/availability-1.svg)


### Solution

Solutions will be posted here at the end of the class period.

## Advanced

Write a program that creates the information in the dictionary `datasets` by querying the SSCWeb HAPI server. That is, write a program that creates the content that is stored in `availability.pkl`. (Metadata queries were covered in [HAPI_02.ipynb#Metadata](HAPI_02.ipynb#Metadata).)

### Solution

Solutions will be posted here at the end of the class period.

# Working with Metadata II.

Starting with

```python
import pickle as pickle
with open('data/availability.pkl', 'rb') as f:
    datasets = pickle.load(f)

start = "2003-10-31T23:00:00Z"
stop = "2003-10-31T23:59:00Z"
```

create a table that indicates the [spacecraft region](https://sscweb.gsfc.nasa.gov/users_guide/ssc_reg_doc.shtml) on `2003-10-31T23:00:00Z` for `ace`, `akebono`, `apex`, and `aqua` from SSCWeb. Your table should have columns of `Spacecraft ID`, `Region`, and `Time`, corresponding to the first time value on or after `start` as shown below.

(Please do not attempt to create a table for all spacecraft - the SSCWeb HAPI server was not designed to handle a class of 300+ and so possibly 10-50 simultaneous requests.)

```
--------------------------------------------------------------------------------
S/C ID           Region        Time

ace              Intpl_Med	 2003-304T23:00:00Z
akebono          D_Psphere	 2003-304T23:00:00Z
apex             D_Msphere	 2003-304T23:00:00Z
aqua             N_Psphere	 2003-304T23:00:00Z
--------------------------------------------------------------------------------
```

## Solution

Solutions will be posted at the end of the class period.

# Working with Data

Many datasets from CDAWeb contain ephemeris (position) data for the associated satellite.

Use https://hapi-server.org/servers/ or https://heliophysicsdata.gsfc.nasa.gov/ to

1. find a CDAWeb dataset that contains the ephemeris of a satellite, and
2. find a SSCWeb dataset that contians the ephemeris of the same satellite.

Then

3. write a program to download the data, and  
4. create a plot that compares the data.

## Solution

A sample solution will be posted at the end of the class period.

# Interpolating Time

For analysis, it is often useful to place two datasets that have different timestamps on the same time grid. 

There are many ways to do this, for example,

* Convert the timestamps in the NumPy `ndarray` returned by `hapi()` into integers, create a 1-D array of time integers to interpolate on to, and then either write your own interpolation function (not recommended) or use an interpolation function in [NumPy](https://numpy.org/doc/stable/reference/generated/numpy.interp.html) or [SciPy](https://docs.scipy.org/doc/scipy/reference/interpolate.html). An example of converting HAPI timestamps to `datetime` objects was given in [HAPI_02.ipynb#Time-Representation](HAPI_02.ipynb#Time-Representation); one can use [`datetime` methods](https://docs.python.org/3/library/datetime.html) to convert `datetime` objects into integers.


* Place data in a Pandas `DataFrame` and use its interpolation methods (an example of placing data into a Pandas `DataFrame` was given in [HAPI_02.ipynb#Convert-to-Pandas-DataFrame](HAPI_02.ipynb#Convert-to-Pandas-DataFrame)).


* In the [SpacePy](https://github.com/heliophysicsPy/summer-school/blob/main/spacepy-tutorial/SpacePy%20-%20MMS%20Ephemeris.md) tutorial (search on `tb.inter`) on Day 2, you used the [`interpol`](https://spacepy.github.io/autosummary/spacepy.toolbox.interpol.html#spacepy.toolbox.interpol) function in [`spacepy.toolbox`](https://spacepy.github.io/toolbox.html) to perform interpolation.

Starting with the following program that reads datasets from two different data servers, use any library (or your own code) to

1. write a program that interpolates `data2` on to the time grid of `data0` and

2. create a table or plot that allows one to visually compare the interpolated values with the given values.

Optionally,

3. Write a program that averages `data0` into 1-hour time bins and compares the result with the contents of `data2`.

In [11]:
from hapiclient import hapi, hapitime2datetime

server     = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset    = 'AC_H0_SWE'
parameters = 'Np'
start      = '1998-02-04'
stop       = '1998-02-06'

data0, meta0 = hapi(server, dataset, parameters, start, stop)
print('data0 = ')
print(data0)

server     = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset    = 'AC_H2_SWE'
parameters = 'Np'
start      = '1998-02-04'
stop       = '1998-02-06'

data2, meta2 = hapi(server, dataset, parameters, start, stop)
print('\ndata2 = ')
print(data2)

data0 = 
[(b'1998-02-04T00:00:31.000Z', -1.00000000e+31)
 (b'1998-02-04T00:01:35.000Z', -1.00000000e+31)
 (b'1998-02-04T00:02:39.000Z', -1.00000000e+31) ...
 (b'1998-02-05T23:57:19.000Z',  1.44819002e+01)
 (b'1998-02-05T23:58:23.000Z',  1.49483004e+01)
 (b'1998-02-05T23:59:27.000Z',  1.35783005e+01)]

data2 = 
[(b'1998-02-04T00:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T01:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T02:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T03:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T04:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T05:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T06:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T07:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T08:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T09:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T10:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T11:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T12:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T13:00:00.000Z', -1.00000000e+31)
 (b'1998-02-04T1

## Solution

A sample solution will be posted at the end of the class period.

# Coordinate Transform

Starting with the following program,

1. Print out the `GSE` and `GSM` values reported by SSCWeb.
2. Use SpacePy to convert the `GSE` values to `GSM`.

In [12]:
import spacepy.coordinates as sc
from spacepy.time import Ticktock

from hapiclient import hapi, hapitime2datetime

server     = 'https://hapi-server.org/servers/SSCWeb/hapi'
dataset    = 'swarma'
parameters = 'X_GSE,Y_GSE,Z_GSE,X_GSM,Y_GSM,Z_GSM'
start      = '2013-11-26T00:00:00Z'
stop       = '2013-11-26T00:01:00Z'

opts       = {'logging': True, 'usecache': True, 'cachedir': './hapicache'}

data, meta = hapi(server, dataset, parameters, start, stop)

## Solution

A sample solution will be posted at the end of the class period.

# SunPy Data Fusion

In the last part of the [SunPy Tutorial](https://github.com/heliophysicsPy/summer-school/blob/main/sunpy-tutorial/05-multi-observer-application.ipynb), you plotted solar images on 2021-04-24. 

Solar wind plasma typically ~4 days to propagate from the solar surface to Earth. 

Find and plot data from at least one spacecraft from `2021-04-24` to `2021-04-30`. Be prepared to answer questions about your interpretation of any features in the time series.

## Solution

A sample solution will be posted at the end of the class period.